# Principle Component Analysis 

## Objectives

### **What** is PCA? 
- Set-up example problem with shipping costs data 
- Walk thru simple PCA problem start to finish 

### **Why** do we need PCA? 
- Discuss advantages/disadvantages of PCA 
- When would you want to use PCA? 

### **How** do we apply PCA to a data problem using sci-kit learn? 
- Apply each step in PCA manually to shipping data 
- Model shipping data with PCA components 
- Walk-thru a second example with cars data and pipelines 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

# Scenario: Shipping Costs
Let's say that we want to predict the cost to ship a package based on its properties.

White board walk-thru...

## Pros, cons and use cases of PCA 


> PCA is a feature extraction technique typically used to reduce dimensionality(Can you think of another one?). It combines our input variables in a way that allows us to drop the 'least important' variables while still retaining the most valuable parts. 

**More features** = More dimensions --> Overfitting 

### When to use it? 
1. When you want to eliminate features but are unsure which ones to remove.  
2. You want to create independent variables.  
3. Interpretability is not a priority. 

In [ ]:
packages = pd.read_csv('data/packages.csv')
packages.head(10)

## Revisting Dimensionality
You can think about each variable as a dimension, and thus each package as a data point. If we take just one feature, we can easily visualize this in 2 dimensional space

In [ ]:
plt.style.use('fivethirtyeight')
packages.plot(kind='scatter', y='Shipping Cost ($)', x='Length (in)');

> You can think of each package as a point in six-dimensional space - 5 dimensions for our features and 1 for our target.

## Remember Correlation and Covariance Matrices?
The first four features in this dataset all relate to package size, so we might expect them to be strongly related.

In [ ]:
sns.heatmap(packages.corr(),
            annot=True,
            fmt='0.2g',
            vmin=-1,
            vmax=1,
            center=0,
            cmap='coolwarm');

PCA does not use this correlation matrix, which is conveniently scaled between -1 and 1. Rather, it uses the covariance matrix, which is scaled in square units of the original variables. This makes PCA very sensitive to the scale of the variables.

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))
ax = sns.heatmap(packages.cov(),
            annot=True,
            fmt='0.2g',
            center=0,
            cmap='coolwarm');

Let's normalize our variables to mean = 0 & SD = 1, which will make our covariance matrix equal the correlation matrix.

In [ ]:
packages_scaled = (packages - packages.mean())/packages.std()
sns.heatmap(packages_scaled.cov(),
            annot=True,
            fmt='0.2g',
            center=0,
            cmap='coolwarm');

Notice that, for the centered data matrix $X$, the covariance matrix $C$ is equal to $\frac{1}{n-1}X^TX$:

In [ ]:
packages_scaled.T.dot(packages_scaled) / (len(packages_scaled)-1)

> That means that the covariance matrix preserves the information about the spread of our dataset. What we want to do now is to explain that spread, one linear transformation (one **eigenvector**) at a time.

Let's try to reduce the dimensionality of our dataset. Since the features capturing size are strongly correlated, we might expect to be able to reduce our feature dimensions down to two without losing much information (i.e. variance in our features).

## Eigendecomposition

We will use an **eigendecomposition** of the covariance matrix to create a new set of dimensions. We can then decide how many of these dimensions to keep based on how much variance is captured by each dimension.

Here, we show you how to do this using the NumPy `.eig()` function, but we will learn how to do PCA more easily in `sklearn` later.

In [ ]:
y_packages_scaled = packages_scaled['Shipping Cost ($)']
X_packages_scaled = packages_scaled.drop('Shipping Cost ($)', axis=1)

cov_mat = X_packages_scaled.cov().values #array of our covariance values 
eigvals, eigvecs = np.linalg.eig(cov_mat)

This decomposition gives us two things: eigenvalues and eigenvectors.

## Eigenvalues
Eigenvalues represent the relative amount of variance captured by each new dimension. The average eigenvalue will be 1, so we look for values over 1 to identify dimensions that capture more variance than average.

In [ ]:
eigvals

> It looks like we have one great dimension capturing 3.4x more variance than average, one OK dimension capturing an average amount of variance, and three other dimensions that don't capture much variance. This is in line with what we were expecting! It means that we can just use the first two dimensions - and drop the last three - without losing much variance/information from our predictors.

### Proportion of Variance
You can also divide your eigenvalues by the number of features and then interpret them as the proportion of variance in the features captured by each dimension.

In [ ]:
eigvals/5

## Eigenvectors (aka Principal Components)
Eigenvectors represent the new dimensions, which we call principal components when doing PCA. There is one eigenvector for each dimension, and they are all combined together into one matrix.

In [ ]:
eigvecs

In PCA, the values in our eigenvectors are called component weights, and they tell us how much variance of each feature is captured by that dimension. These weights range from -1 to 1, but the relative sizes are what matter.

### Orthogonality
These eigenvectors are orthogonal, meaning their dot product is zero. Think of it like being at right angles, like the x and y axes of a graph, but in higher-dimensional space.

In [ ]:
eigvec1 = eigvecs[:, 0]
eigvec2 = eigvecs[:, 1]
eigvec1.dot(eigvec2)

### First Principal Component
The first column of `eigvecs` is our first eigenvector, corresponding to the eigenvalue of 3.4. Let's look at it.

In [ ]:
eigvec1

Notice that the first four numbers are relatively large, while the fifth is near zero. This means that this first dimension is almost entirely capturing the shared variance in our four size features, as we hoped! It's also interesting to note that the weights for the four features are almost equal, so they are equally represented in this dimension.

### Second Principal Component
Let's look at our second eigenvector and see what features it seems to be capturing.

In [ ]:
eigvec2

Looks like it is almost entirely capturing the distance dimension, which makes sense, since that is not related to the package size at all. It has an eigenvalue of 1, which is appropriate, since the eigenvector only captures one feature, which wasn't captured at all in the first principal component.

### Remaining Principal Components
Since the remaining eigenvalues were all much less than 1, we can ignore the eigenvectors associated with them. We will not include components corresponding to them in our model.

## Sidebar: Properties of Eigenvectors
These eigenvectors have **unit length** (length 1) in multi-dimensional space.

In [ ]:
#sum of the sinular values 
np.linalg.norm(eigvec1)

Eigenvectors are related to eigenvalues by the following property: $\vec{x}$ is an eigenvector of the matrix $A$ if $A\vec{x} = \lambda\vec{x}$, for some eigenvalue $\lambda$.

In [ ]:
cov_mat.dot(eigvec1)

In [ ]:
eigval1 = eigvals[0]
eigval1*eigvec1

## Transforming Data
We will now use these principal components to create new features. These features will be weighted sums (aka linear combinations) of existing features, using the component weights from the eigenvectors.

### First Component
We will now create a new feature using the first principal component.

In [ ]:
eigvec1

Our first feature will be calculated as follows:

**PC1** = 0.492 * Length + 0.508 * Width + 0.508 * Height + 0.492 * Weight - 0.003 * Distance

We use a dot product between the data and the eigenvector to do the arithmetic for us.

In [ ]:
data_array = X_packages_scaled.values
pc1 = data_array.dot(eigvec1)
X_packages_pca = pd.DataFrame(data = pc1, columns=['PC1'])
X_packages_pca.head()

### All Components

You can calculate all the new features at once using a dot product with the `eigvecs` matrix, which has all the eigenvectors in it.

In [ ]:
pcs = data_array.dot(eigvecs)
X_packages_pca = pd.DataFrame(data = pcs, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_packages_pca.head(10)

## Feature Correlations
Because we used eigenvectors to construct our new features, we have completely solved any multicollinearity issues. This is because the eigenvectors define new, uncorrelated dimensions:

In [ ]:
sns.heatmap(X_packages_pca.corr(),
            annot=True,
            fmt='0.2g',
            vmin=-1,
            vmax=1,
            center=0,
            cmap= 'coolwarm');

## PCA in `sklearn`

In [ ]:
pca = PCA(n_components=2) # Check out how `n_components` works

X_packages_pca2 = pca.fit_transform(X_packages_scaled)

In [ ]:
#what is this? 
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

Sometimes the signs get flipped on the eigenvectors - don't worry about it. Think of "up" and "down" as both representing the same dimension, just in opposite directions.

In [ ]:
pca.components_

## Modeling
Then you can use your transformed data as you would in any model
Now let's compare linear regression without PCA to linear regression with PCA. 

Note that we are skipping the validation process with these models - we'll do that later.

In [ ]:
#lr no PCA 
lr = LinearRegression()
lr.fit(X_packages_scaled, y_packages_scaled)
lr.score(X_packages_scaled, y_packages_scaled)

In [ ]:
#lr with PCA 
lr_pca = LinearRegression()
lr_pca.fit(X_packages_pca2, y_packages_scaled)
lr_pca.score(X_packages_pca2, y_packages_scaled)

# Scenario: Car Properties
Use PCA to reduce the dimensionality of features in the example below: Predict car mpg using car properties. We've done the data prep. Now you practice the modeling, including scoring on the test set.

## Data Prep

In [ ]:
cars = pd.read_csv('data/cars.csv')

In [ ]:
cars.head()

In [ ]:
cars.info()

In [ ]:
cars[' cubicinches'].replace(' ', np.nan, inplace=True)
cars[' cubicinches'] = cars[' cubicinches'].astype(float)

In [ ]:
cars[' weightlbs'].replace(' ', np.nan, inplace=True)
cars[' weightlbs'] = cars[' weightlbs'].astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cars.drop('mpg', axis=1),
                                                    cars['mpg'],
                                                   random_state=20)

In [ ]:
ct1 = ColumnTransformer(transformers=[
    ('imputer', SimpleImputer(), [1, 3])],
    remainder='passthrough')

In [ ]:
ct2 = ColumnTransformer(transformers=[
    ('scaler', StandardScaler(), [0, 1, 2, 3, 4, 5]),
    ('ohe', OneHotEncoder(), [6])],
    remainder='passthrough')

In [ ]:
pipe = Pipeline(steps=[
    ('ct1', ct1),
    ('ct2', ct2)
])

In [ ]:
pipe.fit(X_train)

In [ ]:
X_tr_pp = pipe.transform(X_train)
X_te_pp = pipe.transform(X_test)

## Your Model

In [ ]:
#fit linear regression model and score on training and testing data 

<details>
    <summary>
        No peeking until you've tried it out first!
    </summary>
<code>
## Let's start with a linear regression
lr = LinearRegression().fit(X_tr_pp, y_train)
## Score on train
lr.score(X_tr_pp, y_train)
## Score on test
lr.score(X_te_pp, y_test)
    </code>
</details>

In [ ]:
# Get the coefficients of the best-fit hyperplane

lr.coef_

Thus, our best-fit hyperplane is given by:

$2.177\times in^3\_sd - 4.645\times lbs.\_sd - 1.555\times cyl\_sd - 1.154\times hp\_sd -  0.267\times time_{60}\_sd + 2.604\times yr\_sd + 0.708\times brand_{Europe} + 0.912\times brand_{Japan} - 1.620\times brand_{US}$

In [ ]:
cars.columns

In [ ]:
cars_pca = PCA(n_components=3) 

X_train_new = cars_pca.fit_transform(X_tr_pp)
X_test_new = cars_pca.transform(X_te_pp)

In [ ]:
cars_pca.components_

The results of our PCA are as follows:

1. **PC1** = 0.465 * cubicinches_sd + 0.435 * weightlbs_sd + 0.449 * cylinders_sd + 0.454 * hp_sd - 0.349 * time-to-60_sd - 0.187 * year_sd - 0.068 * Europe - 0.073 * Japan + 0.140 * US

2. **PC2** = -0.099 * cubicinches_sd - 0.196 * weightlbs_sd - 0.131 * cylinders_sd + 0.006 * hp_sd - 0.125 * time-to-60_sd - 0.937 * year_sd + 0.129 * Europe + 0.022 * Japan - 0.152 * US

3. **PC3** = 0.141 * cubicinches_sd + 0.342 * weightlbs_sd + 0.187 * cylinders_sd - 0.144 * hp_sd + 0.851 * time-to-60_sd - 0.239 * year_sd + 0.043 * Europe - 0.132 * Japan + 0.089 * US

### Modeling with New Dimensions
Now that we have optimized our features, we can build a new model with them!

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_train_new, y_train)
lr_pca.score(X_train_new, y_train)

In [ ]:
X_test_new = cars_pca.transform(X_te_pp)

In [ ]:
lr_pca.score(X_test_new, y_test)

In [ ]:
lr_pca.coef_

Thus, our best-fit hyperplane is given by:

$-2.967\times PC1 - 1.162\times PC2 -2.486\times PC3$

## Reassembling the whole dataset for the sake of visualization


In [ ]:
X_transformed = np.vstack([X_train_new, X_test_new])
y_new = np.concatenate([y_train, y_test])

In [ ]:
df = pd.DataFrame(np.hstack([X_transformed, y_new[:, np.newaxis]]),
                  columns=['PC1', 'PC2', 'PC3', 'y'])
df.head()

In [ ]:
plt.style.use('fivethirtyeight')
sns.relplot(data=df,
            x='PC1',
            y='PC2',
           hue='y');

In [ ]:
plt.plot(np.cumsum(cars_pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(cars_pca.components_, cmap='plasma');

# Review 
1. How does PCA accomplish dimensionality reduction? 
2. When do you want to use PCA? 
3. Which variables have the greatest influence on PCA? 
4. Can you define what a loading; an eigenvalue and an eigenvector are? 